### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [6]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash",  api_key = GEMINI_API_KEY)

### And make some Agents

In [7]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        # model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [8]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [9]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [10]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of choosing AutoGen for an AI Agent project:

*   **Scalability and Customization:** AutoGen's modular design allows for building scalable and highly customized multi-agent systems.
*   **Ease of Use:** Features integrated observability and debugging tools, simplifying monitoring and control of agent workflows.
*   **Multi-Agent Collaboration:** AutoGen facilitates conversation-driven collaboration between agents, improving factuality, reasoning, and problem-solving.
*   **Task Orchestration and Workflow Optimization:** AutoGen automates task orchestration and optimizes workflows in multi-agent systems.
*   **Flexibility:** The framework is highly customizable, allowing users to extend agents with additional components and define custom workflows.
*   **Microsoft Backing:** Being a Microsoft-backed open-source project, AutoGen benefits from ongoing development and a wide range of usage scenarios.



## Cons of AutoGen:
Here are some cons of using AutoGen for an AI Agent project:

*   **Complex Documentation:** The documentation can be difficult to understand and lacks sufficient examples, making it harder to learn and use effectively.
*   **Functionality Issues:** Some features, like structured outputs, may not work as expected.
*   **LLM Limitations:** AutoGen relies on LLMs, which can have limitations in reasoning capabilities.
*   **Cost:** Using LLMs can be expensive, especially with inefficient workflows, and may lead to rate limit issues.
*   **Limited Creativity:** While good at generating content based on templates, AutoGen might lack originality.
*   **Overlapping Functionality:** There are similar applications to AutoGen (e.g., CrewAI), and AutoGen may not sufficiently differentiate itself.
*   **Integration Challenges:** Integrating AutoGen with other tools or frameworks might present challenges.





## Decision:

Decision: Undecided.

Rationale: The pros highlight significant benefits in scalability, customization, and multi-agent collaboration, which are attractive. However, the cons raise concerns about complex documentation, potential functionality issues, LLM limitations (including cost), and integration challenges. More investigation and prototyping are needed to weigh these conflicting points and determine if AutoGen is the right fit for the project, specifically in relation to its documentation and functionality issues.


In [11]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [12]:
await host.stop()